# 쏠2호 이상치 처리

In [6]:
#!pip install pandas_profiling
#!pip install imagehash

In [1]:
import math

import csv
import json
import pandas as pd
import os
import fnmatch
#import pandas_profiling
from pandas import read_csv

capacity = 96.6
base_anomaly = 10000

RSRSID_list=['RSRS0000000239', 'RSRS0000000241', 'RSRS0000000247', 'RSRS0000000249']
file_path = 'C:/elsys/solar/_ONM/csv_'+ RSRSID_list[1]+'/'
file_list = os.listdir(file_path)
for filename in file_list:
    if fnmatch.fnmatch(filename,'_202*.csv'):
        print(file_path+filename)
        filedata = pd.read_csv(file_path+filename, encoding='cp949')
        sol2 = pd.DataFrame(filedata)
        display(sol2)
#origin.to_csv(dir_path+"/weather_2014_2020.csv",mode='w',index=False, encoding='CP949')   

C:/elsys/solar/_ONM/csv_RSRS0000000241/_20200101_20210531.csv


,COMPANY_NM,SET_GEN_NM,GEN_ID,GEN_NM,CBP_GEN_ID,CAPACITY,PATN_DT,HR_01,HR_02,HR_03,...,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23,HR_24
0,(주)엘시스,elsys0001,235,쏠2호태양광발전소,AE97,96.6,2020-01-01,0.0,0.0,0.0,...,25.0,6.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,(주)엘시스,elsys0001,235,쏠2호태양광발전소,AE97,96.6,2020-01-02,0.0,0.0,0.0,...,20.0,10.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,(주)엘시스,elsys0001,235,쏠2호태양광발전소,AE97,96.6,2020-01-03,0.0,0.0,0.0,...,25.0,10.0,6.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,(주)엘시스,elsys0001,235,쏠2호태양광발전소,AE97,96.6,2020-01-04,0.0,0.0,0.0,...,22.0,11.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,(주)엘시스,elsys0001,235,쏠2호태양광발전소,AE97,96.6,2020-01-05,0.0,0.0,0.0,...,23.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
511,(주)엘시스,elsys0001,235,쏠2호태양광발전소,AE97,96.6,2021-05-27,0.0,0.0,0.0,...,19.0,22.0,21.0,13.0,5.0,0.0,0.0,0.0,0.0,0.0
512,(주)엘시스,elsys0001,235,쏠2호태양광발전소,AE97,96.6,2021-05-28,0.0,0.0,0.0,...,29.0,20.0,11.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
513,(주)엘시스,elsys0001,235,쏠2호태양광발전소,AE97,96.6,2021-05-29,0.0,0.0,0.0,...,28.0,27.0,25.0,125145.0,125136.0,125133.0,0.0,0.0,0.0,0.0
514,(주)엘시스,elsys0001,235,쏠2호태양광발전소,AE97,96.6,2021-05-30,0.0,0.0,0.0,...,26.0,25.0,19.0,12.0,3.0,125158.0,0.0,0.0,0.0,0.0


In [2]:
print(sol2.shape)

(516, 31)


In [3]:
df = sol2.copy()

In [4]:
def getFirstAnomaly(base_anomaly):
    candidate_r=0
    candidate_c=0
    for r in range(sol2.shape[0]):
        for c in range(7,df.shape[1]):
            if capacity < df.iloc[r,c]:
                base_anomaly = df.iloc[r,c]
                df.iloc[r,c] = (df.iloc[r,c-1]+df.iloc[r,c+1])/2
                candidate_r=r
                candidate_c=c
                print("base anomaly:{} , df.iloc[{},{}]:{}".format(base_anomaly, r,c,df.iloc[r,c]))
                return base_anomaly, candidate_r
            
base_anomaly, candidate_r = getFirstAnomaly(base_anomaly)

base anomaly:103.0 , df.iloc[88,22]:24.5


In [26]:
# 저녁부터 체크
  
print(df.shape)
cnt_processed = 0
cnt_nonprocessed = 0
print("for range ({},{},{})".format(df.shape[1]-1, 24-1, -1))

for r in range(candidate_r, df.shape[0]):
    anomal_exist_yn = 'N'
    for c in range(df.shape[1]-1, 24-1, -1):# 17PM~
        
        if df.iloc[r,c] > capacity:
            if df.iloc[r,c+1] > 0:# 오른쪽 0 아님
                if df.iloc[r,c-1] < capacity:
                    base_anomaly = df.iloc[r,c]-math.floor((df.iloc[r,c+1]+df.iloc[r,c-1])/2)
                else:
                    base_anomaly = df.iloc[r,c]-(df.iloc[r,c+1]+1)
            else:# 오른쪽 0
                if df.iloc[r,c-1] < capacity:
                    base_anomaly = df.iloc[r,c]-math.floor((df.iloc[r,c+1]+df.iloc[r,c-1])/2)
                else:
                    base_anomaly = df.iloc[r,c]-1
                
            anomal_exist_yn = 'Y'
            break;
            
    
    for c in range(df.shape[1]-1, 7-1, -1):
        if capacity < df.iloc[r,c]:
            if(anomal_exist_yn == 'Y'):
                cnt_processed = cnt_processed+1
                df.iloc[r,c] = df.iloc[r,c]-base_anomaly
            else:
                cnt_nonprocessed = cnt_nonprocessed+1
                

df.to_csv(file_path+'_sol2_AnomalyProcessing_pm2.csv',mode='w',index=False, encoding='CP949')   
print("Anomaly_processed O {} ".format(cnt_processed)) 
print("Anomaly_processed X {} ".format(cnt_nonprocessed))

'''            
candidate_idx=[7,8,9,10,11,12,13,22,23,24,25,26,27,28,29,30,31]
'''

(516, 31)
for range (30,23,-1)
Anomaly_processed O 191 
Anomaly_processed X 365 


'            \ncandidate_idx=[7,8,9,10,11,12,13,22,23,24,25,26,27,28,29,30,31]\n'

In [27]:
def getFirstAnomaly(base_anomaly):
    candidate_r=0
    candidate_c=0
    for r in range(sol2.shape[0]):
        for c in range(7,df.shape[1]):
            if capacity < df.iloc[r,c]:
                base_anomaly = df.iloc[r,c]
                df.iloc[r,c] = (df.iloc[r,c-1]+df.iloc[r,c+1])/2
                candidate_r=r
                candidate_c=c
                print("base anomaly:{} , df.iloc[{},{}]:{}".format(base_anomaly, r,c,df.iloc[r,c]))
                return base_anomaly, candidate_r
            
base_anomaly, candidate_r = getFirstAnomaly(base_anomaly)

base anomaly:120740.0 , df.iloc[286,20]:13.0


In [28]:
# 아침부터 체크
       
print(df.shape)
cnt_processed = 0
cnt_nonprocessed = 0
print("for range ({},{},{})".format(df.shape[1]-1, 24-1, -1))

for r in range(candidate_r, df.shape[0]):
    anomal_exist_yn = 'N'
    for c in range(7, 16):# ~8AM
        
        if df.iloc[r,c] > capacity:
            if df.iloc[r,c-1] > 0:# 왼쪽 0 아님
                if df.iloc[r,c+1] < capacity:
                    base_anomaly = df.iloc[r,c]-math.floor((df.iloc[r,c-1]+df.iloc[r,c+1])/2)
                else:
                    base_anomaly = df.iloc[r,c]-(df.iloc[r,c-1]+1)
            else:# 왼쪽 0
                if df.iloc[r,c+1] < capacity:
                    base_anomaly = df.iloc[r,c]-math.floor((df.iloc[r,c+1]+df.iloc[r,c-1])/2)
                else:
                    base_anomaly = df.iloc[r,c]-1
                
            anomal_exist_yn = 'Y'
            break;
            
    for c in range(7, df.shape[1]):
        if capacity < df.iloc[r,c]:
            if(anomal_exist_yn == 'Y'):
                cnt_processed = cnt_processed+1
                df.iloc[r,c] = df.iloc[r,c]-base_anomaly
            else:
                cnt_nonprocessed = cnt_nonprocessed+1
                

df.to_csv(file_path+'_sol2_AnomalyProcessing_am2.csv',mode='w',index=False, encoding='CP949')   
print("Anomaly_processed O {} ".format(cnt_processed)) 
print("Anomaly_processed X {} ".format(cnt_nonprocessed))

(516, 31)
for range (30,23,-1)
Anomaly_processed O 175 
Anomaly_processed X 189 


# ------------------------------------------------------------------------------------------------------------KPX power
# -------------------------------------------------------------------------------------------------------------

### 통합

In [69]:
import csv
import json
import pandas as pd
import os
import glob
import datetime
# .json -> .csv


RSRSID_list=['RSRS0000000239', 'RSRS0000000241', 'RSRS0000000247', 'RSRS0000000249']
err_file_size = []

for RSRSID in RSRSID_list:

    base_dir        = 'C:/elsys/solar/_KPX/json_'+ RSRSID
    save_dir        = 'C:/elsys/solar/_KPX/csv_'+ RSRSID+'/'
    print(base_dir)
    
    column_list=['COMPANY_NM','SET_GEN_NM','GEN_ID','GEN_NM','CBP_GEN_ID','CAPACITY','PATN_DT','HR_01','HR_02','HR_03','HR_04','HR_05','HR_06','HR_07','HR_08','HR_09','HR_10','HR_11','HR_12','HR_13','HR_14','HR_15','HR_16','HR_17','HR_18','HR_19','HR_20','HR_21','HR_22','HR_23','HR_24']
    COMPANY_NM=[]
    PATN_DT=[]
    GEN_NM=[]
    CBP_GEN_ID=[]
    SET_GEN_NM=[]
    CAPACITY=[]
    GEN_ID=[]
    HR_01=[]
    HR_02=[]
    HR_03=[]
    HR_04=[]
    HR_05=[]
    HR_06=[]
    HR_07=[]
    HR_08=[]
    HR_09=[]
    HR_10=[]
    HR_11=[]
    HR_12=[]
    HR_13=[]
    HR_14=[]
    HR_15=[]
    HR_16=[]
    HR_17=[]
    HR_18=[]
    HR_19=[]
    HR_20=[]
    HR_21=[]
    HR_22=[]
    HR_23=[]
    HR_24=[]

    json_pattern    = os.path.join(base_dir,'*.json')
    file_list       = glob.glob(json_pattern)
    #print(file_list)

    for file in file_list:
        with open(file, encoding='UTF-8') as json_file:
            #print(file)
            

            # parsing json file
            json_data = json.load(json_file)
            data = json_data['result']
            
            
            # 2. empty file check
            file_size = os.path.getsize(file)
            #print(file_size)
            if file_size < 100:
                err_file_size.append(file + "/" + str(file_size))#err2
                continue
                
                
            COMPANY_NM.append(data['COMPANY_NM'])
            SET_GEN_NM.append(data['SET_GEN_NM'])
            GEN_ID.append(data['GEN_ID'])
            GEN_NM.append(data['GEN_NM'])
            CBP_GEN_ID.append(data['CBP_GEN_ID'])
            CAPACITY.append(data['CAPACITY'])
            PATN_DT.append(data['PATN_DT'])
            HR_01.append(data['HR_01'])
            HR_02.append(data['HR_02'])
            HR_03.append(data['HR_03'])
            HR_04.append(data['HR_04'])
            HR_05.append(data['HR_05'])
            HR_06.append(data['HR_06'])
            HR_07.append(data['HR_07'])
            HR_08.append(data['HR_08'])
            HR_09.append(data['HR_09'])
            HR_10.append(data['HR_10'])
            HR_11.append(data['HR_11'])
            HR_12.append(data['HR_12'])
            HR_13.append(data['HR_13'])
            HR_14.append(data['HR_14'])
            HR_15.append(data['HR_15'])
            HR_16.append(data['HR_16'])
            HR_17.append(data['HR_17'])
            HR_18.append(data['HR_18'])
            HR_19.append(data['HR_19'])
            HR_20.append(data['HR_20'])
            HR_21.append(data['HR_21'])
            HR_22.append(data['HR_22'])
            HR_23.append(data['HR_23'])
            HR_24.append(data['HR_24'])
                    
    df = pd.DataFrame(columns=column_list)
    df['COMPANY_NM'] = COMPANY_NM
    df['SET_GEN_NM'] = SET_GEN_NM
    df['GEN_ID'] = GEN_ID
    df['GEN_NM'] = GEN_NM
    df['CBP_GEN_ID'] = CBP_GEN_ID
    df['CAPACITY'] = CAPACITY
    df['PATN_DT'] = PATN_DT
    df['HR_01'] = HR_01
    df['HR_02'] = HR_02
    df['HR_03'] = HR_03
    df['HR_04'] = HR_04
    df['HR_05'] = HR_05
    df['HR_06'] = HR_06
    df['HR_07'] = HR_07
    df['HR_08'] = HR_08
    df['HR_09'] = HR_09
    df['HR_10'] = HR_10
    df['HR_11'] = HR_11
    df['HR_12'] = HR_12
    df['HR_13'] = HR_13
    df['HR_14'] = HR_14
    df['HR_15'] = HR_15
    df['HR_16'] = HR_16
    df['HR_17'] = HR_17
    df['HR_18'] = HR_18
    df['HR_19'] = HR_19
    df['HR_20'] = HR_20
    df['HR_21'] = HR_21
    df['HR_22'] = HR_22
    df['HR_23'] = HR_23
    df['HR_24'] = HR_24
    df.sort_values('PATN_DT')
    
    # save csv file
    savefilenm = '_' + df.iloc[0,6].replace("-","") + '_' + df.iloc[-1,6].replace("-","") + '.csv'
    df.to_csv(save_dir + savefilenm, index=False, header=True, encoding='CP949')
    
print("-----------------------------------------------------------------")
print("\t FILE SIZE ERROR > file path / file size")
for f in err_file_size:
    print(f)


C:/elsys/solar/_KPX/json_RSRS0000000239
C:/elsys/solar/_KPX/json_RSRS0000000241
C:/elsys/solar/_KPX/json_RSRS0000000247
C:/elsys/solar/_KPX/json_RSRS0000000249
-----------------------------------------------------------------
	 FILE SIZE ERROR > file path / file size


### 개별

In [67]:
import csv
import json
import pandas as pd
import os
import glob
import datetime
# .json -> .csv


RSRSID_list=['RSRS0000000239', 'RSRS0000000241', 'RSRS0000000247', 'RSRS0000000249']
err_file_size = []

for RSRSID in RSRSID_list:

    base_dir        = 'C:/elsys/solar/_KPX/json_'+ RSRSID
    save_dir        = 'C:/elsys/solar/_KPX/csv_'+ RSRSID+'/'
    print(base_dir)
    
    column_list=['COMPANY_NM','SET_GEN_NM','GEN_ID','GEN_NM','CBP_GEN_ID','CAPACITY','PATN_DT','HR_01','HR_02','HR_03','HR_04','HR_05','HR_06','HR_07','HR_08','HR_09','HR_10','HR_11','HR_12','HR_13','HR_14','HR_15','HR_16','HR_17','HR_18','HR_19','HR_20','HR_21','HR_22','HR_23','HR_24']
    COMPANY_NM=[]
    PATN_DT=[]
    GEN_NM=[]
    CBP_GEN_ID=[]
    SET_GEN_NM=[]
    CAPACITY=[]
    GEN_ID=[]
    HR_01=[]
    HR_02=[]
    HR_03=[]
    HR_04=[]
    HR_05=[]
    HR_06=[]
    HR_07=[]
    HR_08=[]
    HR_09=[]
    HR_10=[]
    HR_11=[]
    HR_12=[]
    HR_13=[]
    HR_14=[]
    HR_15=[]
    HR_16=[]
    HR_17=[]
    HR_18=[]
    HR_19=[]
    HR_20=[]
    HR_21=[]
    HR_22=[]
    HR_23=[]
    HR_24=[]

    json_pattern    = os.path.join(base_dir,'*.json')
    file_list       = glob.glob(json_pattern)
    #print(file_list)

    for file in file_list:
        with open(file, encoding='UTF-8') as json_file:
            #print(file)
            

            # parsing json file
            json_data = json.load(json_file)
            data = json_data['result']
            
            
            # 2. empty file check
            file_size = os.path.getsize(file)
            #print(file_size)
            if file_size < 100:
                err_file_size.append(file + "/" + str(file_size))#err2
                continue
                
                
            
            df['COMPANY_NM']=data['COMPANY_NM']
            df['SET_GEN_NM']=data['SET_GEN_NM']
            df['GEN_ID']=data['GEN_ID']
            df['GEN_NM']=data['GEN_NM']
            df['CBP_GEN_ID']=data['CBP_GEN_ID']
            df['CAPACITY']=data['CAPACITY']
            df['PATN_DT']=data['PATN_DT']
            df['HR_01']=data['HR_01']
            df['HR_02']=data['HR_02']
            df['HR_03']=data['HR_03']
            df['HR_04']=data['HR_04']
            df['HR_05']=data['HR_05']
            df['HR_06']=data['HR_06']
            df['HR_07']=data['HR_07']
            df['HR_08']=data['HR_08']
            df['HR_09']=data['HR_09']
            df['HR_10']=data['HR_10']
            df['HR_11']=data['HR_11']
            df['HR_12']=data['HR_12']
            df['HR_13']=data['HR_13']
            df['HR_14']=data['HR_14']
            df['HR_15']=data['HR_15']
            df['HR_16']=data['HR_16']
            df['HR_17']=data['HR_17']
            df['HR_18']=data['HR_18']
            df['HR_19']=data['HR_19']
            df['HR_20']=data['HR_20']
            df['HR_21']=data['HR_21']
            df['HR_22']=data['HR_22']
            df['HR_23']=data['HR_23']
            df['HR_24']=data['HR_24']
            
            df.sort_values('PATN_DT')

            # save csv file
            df.to_csv(save_dir + file[-13:-5] + '.csv', index=False, header=True, encoding='CP949')
    
print("-----------------------------------------------------------------")
print("\t FILE SIZE ERROR > file path / file size")
for f in err_file_size:
    print(f)


C:/elsys/solar/_KPX/json_RSRS0000000239
C:/elsys/solar/_KPX/json_RSRS0000000241
C:/elsys/solar/_KPX/json_RSRS0000000247
C:/elsys/solar/_KPX/json_RSRS0000000249
-----------------------------------------------------------------
	 FILE SIZE ERROR > file path / file size


# ------------------------------------------------------------------------------------------------------------# OMN power
# -------------------------------------------------------------------------------------------------------------

### 개별 csv

In [66]:
import csv
import json
import pandas as pd
import os
import glob
import datetime
# .json -> .csv


RSRSID_list=['RSRS0000000239', 'RSRS0000000241', 'RSRS0000000247', 'RSRS0000000249']
err_file_size = []

for RSRSID in RSRSID_list:

    base_dir        = 'C:/elsys/solar/_ONM/json_'+ RSRSID
    save_dir        = 'C:/elsys/solar/_ONM/csv_'+ RSRSID+'/'
    print(base_dir)
    
    column_list=['COMPANY_NM','SET_GEN_NM','GEN_ID','GEN_NM','CBP_GEN_ID','CAPACITY','PATN_DT','HR_01','HR_02','HR_03','HR_04','HR_05','HR_06','HR_07','HR_08','HR_09','HR_10','HR_11','HR_12','HR_13','HR_14','HR_15','HR_16','HR_17','HR_18','HR_19','HR_20','HR_21','HR_22','HR_23','HR_24']
    COMPANY_NM=[]
    PATN_DT=[]
    GEN_NM=[]
    CBP_GEN_ID=[]
    SET_GEN_NM=[]
    CAPACITY=[]
    GEN_ID=[]
    HR_01=[]
    HR_02=[]
    HR_03=[]
    HR_04=[]
    HR_05=[]
    HR_06=[]
    HR_07=[]
    HR_08=[]
    HR_09=[]
    HR_10=[]
    HR_11=[]
    HR_12=[]
    HR_13=[]
    HR_14=[]
    HR_15=[]
    HR_16=[]
    HR_17=[]
    HR_18=[]
    HR_19=[]
    HR_20=[]
    HR_21=[]
    HR_22=[]
    HR_23=[]
    HR_24=[]

    json_pattern    = os.path.join(base_dir,'*.json')
    file_list       = glob.glob(json_pattern)
    #print(file_list)

    for file in file_list:
        with open(file, encoding='UTF-8') as json_file:
            #print(file)
            

            # parsing json file
            json_data = json.load(json_file)
            data = json_data['result']
            
            
            # 2. empty file check
            file_size = os.path.getsize(file)
            #print(file_size)
            if file_size < 100:
                err_file_size.append(file + "/" + str(file_size))#err2
                continue
                
                
            
            df['COMPANY_NM']=data['COMPANY_NM']
            df['SET_GEN_NM']=data['SET_GEN_NM']
            df['GEN_ID']=data['GEN_ID']
            df['GEN_NM']=data['GEN_NM']
            df['CBP_GEN_ID']=data['CBP_GEN_ID']
            df['CAPACITY']=data['CAPACITY']
            df['PATN_DT']=data['PATN_DT']
            df['HR_01']=data['HR_01']
            df['HR_02']=data['HR_02']
            df['HR_03']=data['HR_03']
            df['HR_04']=data['HR_04']
            df['HR_05']=data['HR_05']
            df['HR_06']=data['HR_06']
            df['HR_07']=data['HR_07']
            df['HR_08']=data['HR_08']
            df['HR_09']=data['HR_09']
            df['HR_10']=data['HR_10']
            df['HR_11']=data['HR_11']
            df['HR_12']=data['HR_12']
            df['HR_13']=data['HR_13']
            df['HR_14']=data['HR_14']
            df['HR_15']=data['HR_15']
            df['HR_16']=data['HR_16']
            df['HR_17']=data['HR_17']
            df['HR_18']=data['HR_18']
            df['HR_19']=data['HR_19']
            df['HR_20']=data['HR_20']
            df['HR_21']=data['HR_21']
            df['HR_22']=data['HR_22']
            df['HR_23']=data['HR_23']
            df['HR_24']=data['HR_24']
            
            df.sort_values('PATN_DT')

            # save csv file
            df.to_csv(save_dir + file[-13:-5] + '.csv', index=False, header=True, encoding='CP949')
    
print("-----------------------------------------------------------------")
print("\t FILE SIZE ERROR > file path / file size")
for f in err_file_size:
    print(f)


C:/elsys/solar/_OMN/json_RSRS0000000239
C:/elsys/solar/_OMN/json_RSRS0000000241
C:/elsys/solar/_OMN/json_RSRS0000000247
C:/elsys/solar/_OMN/json_RSRS0000000249
-----------------------------------------------------------------
	 FILE SIZE ERROR > file path / file size
C:/elsys/solar/_OMN/json_RSRS0000000241\20201227.json/15
C:/elsys/solar/_OMN/json_RSRS0000000247\20201227.json/15
C:/elsys/solar/_OMN/json_RSRS0000000247\20201228.json/15
C:/elsys/solar/_OMN/json_RSRS0000000249\20201227.json/15


### 통합 csv

In [65]:
import csv
import json
import pandas as pd
import os
import glob
import datetime
# .json -> .csv


RSRSID_list=['RSRS0000000239', 'RSRS0000000241', 'RSRS0000000247', 'RSRS0000000249']
err_file_size = []

for RSRSID in RSRSID_list:

    base_dir        = 'C:/elsys/solar/_ONM/json_'+ RSRSID
    save_dir        = 'C:/elsys/solar/_ONM/csv_'+ RSRSID+'/'
    print(base_dir)
    
    column_list=['COMPANY_NM','SET_GEN_NM','GEN_ID','GEN_NM','CBP_GEN_ID','CAPACITY','PATN_DT','HR_01','HR_02','HR_03','HR_04','HR_05','HR_06','HR_07','HR_08','HR_09','HR_10','HR_11','HR_12','HR_13','HR_14','HR_15','HR_16','HR_17','HR_18','HR_19','HR_20','HR_21','HR_22','HR_23','HR_24']
    COMPANY_NM=[]
    PATN_DT=[]
    GEN_NM=[]
    CBP_GEN_ID=[]
    SET_GEN_NM=[]
    CAPACITY=[]
    GEN_ID=[]
    HR_01=[]
    HR_02=[]
    HR_03=[]
    HR_04=[]
    HR_05=[]
    HR_06=[]
    HR_07=[]
    HR_08=[]
    HR_09=[]
    HR_10=[]
    HR_11=[]
    HR_12=[]
    HR_13=[]
    HR_14=[]
    HR_15=[]
    HR_16=[]
    HR_17=[]
    HR_18=[]
    HR_19=[]
    HR_20=[]
    HR_21=[]
    HR_22=[]
    HR_23=[]
    HR_24=[]

    json_pattern    = os.path.join(base_dir,'*.json')
    file_list       = glob.glob(json_pattern)
    #print(file_list)

    for file in file_list:
        with open(file, encoding='UTF-8') as json_file:
            #print(file)
            

            # parsing json file
            json_data = json.load(json_file)
            data = json_data['result']
            
            
            # 2. empty file check
            file_size = os.path.getsize(file)
            #print(file_size)
            if file_size < 100:
                err_file_size.append(file + "/" + str(file_size))#err2
                continue
                
                
            COMPANY_NM.append(data['COMPANY_NM'])
            SET_GEN_NM.append(data['SET_GEN_NM'])
            GEN_ID.append(data['GEN_ID'])
            GEN_NM.append(data['GEN_NM'])
            CBP_GEN_ID.append(data['CBP_GEN_ID'])
            CAPACITY.append(data['CAPACITY'])
            PATN_DT.append(data['PATN_DT'])
            HR_01.append(data['HR_01'])
            HR_02.append(data['HR_02'])
            HR_03.append(data['HR_03'])
            HR_04.append(data['HR_04'])
            HR_05.append(data['HR_05'])
            HR_06.append(data['HR_06'])
            HR_07.append(data['HR_07'])
            HR_08.append(data['HR_08'])
            HR_09.append(data['HR_09'])
            HR_10.append(data['HR_10'])
            HR_11.append(data['HR_11'])
            HR_12.append(data['HR_12'])
            HR_13.append(data['HR_13'])
            HR_14.append(data['HR_14'])
            HR_15.append(data['HR_15'])
            HR_16.append(data['HR_16'])
            HR_17.append(data['HR_17'])
            HR_18.append(data['HR_18'])
            HR_19.append(data['HR_19'])
            HR_20.append(data['HR_20'])
            HR_21.append(data['HR_21'])
            HR_22.append(data['HR_22'])
            HR_23.append(data['HR_23'])
            HR_24.append(data['HR_24'])
                    
    df = pd.DataFrame(columns=column_list)
    df['COMPANY_NM'] = COMPANY_NM
    df['SET_GEN_NM'] = SET_GEN_NM
    df['GEN_ID'] = GEN_ID
    df['GEN_NM'] = GEN_NM
    df['CBP_GEN_ID'] = CBP_GEN_ID
    df['CAPACITY'] = CAPACITY
    df['PATN_DT'] = PATN_DT
    df['HR_01'] = HR_01
    df['HR_02'] = HR_02
    df['HR_03'] = HR_03
    df['HR_04'] = HR_04
    df['HR_05'] = HR_05
    df['HR_06'] = HR_06
    df['HR_07'] = HR_07
    df['HR_08'] = HR_08
    df['HR_09'] = HR_09
    df['HR_10'] = HR_10
    df['HR_11'] = HR_11
    df['HR_12'] = HR_12
    df['HR_13'] = HR_13
    df['HR_14'] = HR_14
    df['HR_15'] = HR_15
    df['HR_16'] = HR_16
    df['HR_17'] = HR_17
    df['HR_18'] = HR_18
    df['HR_19'] = HR_19
    df['HR_20'] = HR_20
    df['HR_21'] = HR_21
    df['HR_22'] = HR_22
    df['HR_23'] = HR_23
    df['HR_24'] = HR_24
    df.sort_values('PATN_DT')
    
    # save csv file
    savefilenm = '_' + df.iloc[0,6].replace("-","") + '_' + df.iloc[-1,6].replace("-","") + '.csv'
    df.to_csv(save_dir + savefilenm, index=False, header=True, encoding='CP949')
    
print("-----------------------------------------------------------------")
print("\t FILE SIZE ERROR > file path / file size")
for f in err_file_size:
    print(f)


C:/elsys/solar/_OMN/json_RSRS0000000239
C:/elsys/solar/_OMN/json_RSRS0000000241
C:/elsys/solar/_OMN/json_RSRS0000000247
C:/elsys/solar/_OMN/json_RSRS0000000249
-----------------------------------------------------------------
	 FILE SIZE ERROR > file path / file size
C:/elsys/solar/_OMN/json_RSRS0000000241\20201227.json/15
C:/elsys/solar/_OMN/json_RSRS0000000247\20201227.json/15
C:/elsys/solar/_OMN/json_RSRS0000000247\20201228.json/15
C:/elsys/solar/_OMN/json_RSRS0000000249\20201227.json/15


### daily, 1 hour

In [13]:
date_start = '20210129'
date_end   = '20210531'
save_name  =  date_start+'_'+date_end
base_dir   = 'C:/elsys/solar/'
RSRSID_list=['RSRS0000000239', 'RSRS0000000241', 'RSRS0000000247', 'RSRS0000000249']
RSRD_NM = RSRSID_list[0]

# ONM csv 통합
onm_path = base_dir+'_ONM/csv_'+RSRD_NM

file_list   = os.listdir(onm_path)
print(len(file_list))
onm_df = pd.DataFrame()

for filename in file_list:
    if fnmatch.fnmatch(filename, '_*_*.csv'):
        onm_df = pd.read_csv(onm_path+'/'+filename, encoding='CP949')
        print(filename)
        break;

print(onm_df.columns)
display(onm_df)
onm_df.to_csv(base_dir+"/_"+RSRD_NM[11:14]+"_onm_df_"+save_name+"_daily.csv",mode='w',index=False, encoding='CP949')

# 설비용량
capacity = onm_df['CAPACITY'].iloc[0]
print(capacity)


onm_df['NEW_DATE'] = onm_df['PATN_DT'].str[0:4]+onm_df['PATN_DT'].str[5:7]+onm_df['PATN_DT'].str[8:10]
onm_df = onm_df[(onm_df['NEW_DATE'].astype(str).str[0:10]>=date_start) & (onm_df['NEW_DATE'].astype(str).str[0:10]<=date_end)]
onm_df.insert(0, 'DATE', onm_df['NEW_DATE'].values, True)
onm_df = onm_df.drop(['COMPANY_NM', 'SET_GEN_NM', 'GEN_ID', 'GEN_NM', 'CBP_GEN_ID','CAPACITY','NEW_DATE','PATN_DT'], axis=1)

display(onm_df)

518
_20200101_20210531.csv
Index(['COMPANY_NM', 'SET_GEN_NM', 'GEN_ID', 'GEN_NM', 'CBP_GEN_ID',
       'CAPACITY', 'PATN_DT', 'HR_01', 'HR_02', 'HR_03', 'HR_04', 'HR_05',
       'HR_06', 'HR_07', 'HR_08', 'HR_09', 'HR_10', 'HR_11', 'HR_12', 'HR_13',
       'HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_20', 'HR_21',
       'HR_22', 'HR_23', 'HR_24'],
      dtype='object')


,COMPANY_NM,SET_GEN_NM,GEN_ID,GEN_NM,CBP_GEN_ID,CAPACITY,PATN_DT,HR_01,HR_02,HR_03,...,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23,HR_24
0,(주)엘시스,elsys0001,234,쏠태양광발전소,AE96,89.7,2020-01-01,0.0,0.0,0.0,...,5.5,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,(주)엘시스,elsys0001,234,쏠태양광발전소,AE96,89.7,2020-01-02,0.0,0.0,0.0,...,7.1,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,(주)엘시스,elsys0001,234,쏠태양광발전소,AE96,89.7,2020-01-03,0.0,0.0,0.0,...,6.6,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,(주)엘시스,elsys0001,234,쏠태양광발전소,AE96,89.7,2020-01-04,0.0,0.0,0.0,...,7.0,0.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,(주)엘시스,elsys0001,234,쏠태양광발전소,AE96,89.7,2020-01-05,0.0,0.0,0.0,...,8.9,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,(주)엘시스,elsys0001,234,쏠태양광발전소,AE96,89.7,2021-05-27,0.0,0.0,0.0,...,48.6,61.2,44.0,20.2,6.5,0.5,0.0,0.0,0.0,0.0
513,(주)엘시스,elsys0001,234,쏠태양광발전소,AE96,89.7,2021-05-28,0.0,0.0,0.0,...,2.1,51.0,38.1,22.7,5.7,0.4,0.0,0.0,0.0,0.0
514,(주)엘시스,elsys0001,234,쏠태양광발전소,AE96,89.7,2021-05-29,0.0,0.0,0.0,...,65.7,47.1,30.0,20.9,6.5,0.4,0.0,0.0,0.0,0.0
515,(주)엘시스,elsys0001,234,쏠태양광발전소,AE96,89.7,2021-05-30,0.0,0.0,0.0,...,62.4,57.4,40.7,19.8,6.7,0.6,0.0,0.0,0.0,0.0


89.7


,DATE,HR_01,HR_02,HR_03,HR_04,HR_05,HR_06,HR_07,HR_08,HR_09,...,HR_15,HR_16,HR_17,HR_18,HR_19,HR_20,HR_21,HR_22,HR_23,HR_24
394,20210129,0.0,0.0,0.0,0.0,0.0,0.6,10.4,43.3,60.9,...,24.3,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
395,20210130,0.0,0.0,0.0,0.0,0.0,0.3,8.4,33.8,50.1,...,14.2,1.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
396,20210131,0.0,0.0,0.0,0.0,0.0,0.5,13.7,38.4,51.8,...,11.9,1.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
397,20210201,0.0,0.0,0.0,0.0,0.0,0.0,0.9,9.0,17.4,...,3.3,1.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
398,20210202,0.0,0.0,0.0,0.0,0.0,0.6,18.7,50.2,62.0,...,30.4,1.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,20210527,0.0,0.0,0.0,0.0,0.0,0.0,2.7,5.3,10.9,...,48.6,61.2,44.0,20.2,6.5,0.5,0.0,0.0,0.0,0.0
513,20210528,0.0,0.0,0.0,0.0,0.0,0.6,5.4,22.4,43.7,...,2.1,51.0,38.1,22.7,5.7,0.4,0.0,0.0,0.0,0.0
514,20210529,0.0,0.0,0.0,0.0,0.0,0.7,4.8,26.0,46.1,...,65.7,47.1,30.0,20.9,6.5,0.4,0.0,0.0,0.0,0.0
515,20210530,0.0,0.0,0.0,0.0,0.0,0.8,6.6,23.0,37.2,...,62.4,57.4,40.7,19.8,6.7,0.6,0.0,0.0,0.0,0.0


In [14]:
onm_col_list = onm_df.columns
onm_col_list = onm_col_list[1:]
for col in onm_col_list:
    print(col[-2:]+'\',\'', end='')
    
onm_df2 = pd.DataFrame(columns=['DATE','POWER'])
print(onm_df.shape)

for r in range(onm_df.shape[0]):
    #print(onm_df.iloc[r,0]+' ', end='')
    for c in range(1,onm_df.shape[1]):
        IDX = r*len(onm_col_list)+c-1
        TIME = onm_df.iloc[r,0]+onm_col_list[c-1][-2:]
        POWER = onm_df.iloc[r,c]
        onm_df2.loc[IDX] = [TIME, POWER]
        
display(onm_df2)      
onm_df2.to_csv(base_dir+"/_"+RSRD_NM[11:14]+"_onm_df_"+save_name+"_1hour.csv",mode='w',index=False, encoding='CP949')

01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','(123, 25)


,DATE,POWER
0,2021012901,0.0
1,2021012902,0.0
2,2021012903,0.0
3,2021012904,0.0
4,2021012905,0.0
...,...,...
2947,2021053120,0.6
2948,2021053121,0.0
2949,2021053122,0.0
2950,2021053123,0.0


# ------------------------------------------------------------------------------------------------------------- Sensor data
# -------------------------------------------------------------------------------------------------------------
id
loc_oid
TIME
solarradiation
uv
temp
humidity
winddir
windspeed
windgust
dewpoint
maxdailygust
feelslike
hourlyrainin
dailyrainin
weeklyrainin
monthlyrainin
yearlyrainin

### 통합 csv (1시간간격 max)

In [11]:
# 1시간 단위

import csv
import json
import pandas as pd
import os
import glob
import datetime
import numpy as np
# .json -> .csv


base_dir        = 'C:/elsys/solar/'
sensor_dir      = base_dir + '_SENSOR/json'
save_dir        = base_dir + '_SENSOR/csv/'

column_list     = ['id','loc_oid','TIME','solarradiation','uv','temp','humidity','winddir','windspeed','windgust','dewpoint','maxdailygust','feelslike','hourlyrainin','dailyrainin','weeklyrainin','monthlyrainin','yearlyrainin']
err_file_size = []

json_pattern    = os.path.join(sensor_dir,'*.json')
file_list       = glob.glob(json_pattern)

#df = pd.DataFrame(columns=column_list)
total_df = pd.DataFrame(columns=column_list)
for file in file_list:
    with open(file) as json_file:
        print(file)
        # parsing json file
        json_data = json.load(json_file)
        data = json_data['result']
        df = pd.DataFrame(columns=column_list)
        
        # 2. empty file check
        file_size = os.path.getsize(file)
        print(str(file_size)+'\t',end='')
        if file_size < 100:
            err_file_size.append(file + "/" + str(file_size))#err2
            continue
            
        __id              = []
        _loc_oid          = []
        _TIME             = []
        _solarradiation   = []
        _uv               = []
        _temp             = []
        _humidity         = []
        _winddir          = []
        _windspeed        = []
        _windgust         = []
        _dewpoint         = []
        _maxdailygust     = []
        _feelslike        = []
        _hourlyrainin     = []
        _dailyrainin      = []
        _weeklyrainin     = []
        _monthlyrainin    = []
        _yearlyrainin     = []
        
        _id              = []
        loc_oid          = []
        TIME             = []
        solarradiation   = []
        uv               = []
        temp             = []
        humidity         = []
        winddir          = []
        windspeed        = []
        windgust         = []
        dewpoint         = []
        maxdailygust     = []
        feelslike        = []
        hourlyrainin     = []
        dailyrainin      = []
        weeklyrainin     = []
        monthlyrainin    = []
        yearlyrainin     = []
        
        base_time = '00'
        for dataline in data:
            #print("base time: ", base_time, ", ", dataline['TIME'][11:13])
            #print("len(data): ",len(data))
            if base_time != dataline['TIME'][11:13]:
                base_time = dataline['TIME'][11:13]
                if len(__id) != 0:
                    _TIME = dataline['TIME'][:10]+' '+base_time
                    #base_time = dataline['TIME'][11:13]
                    _id.append(max(__id))
                    loc_oid.append(max(_loc_oid))
                    TIME.append(_TIME)
                    solarradiation.append(max(_solarradiation))
                    uv.append(max(_uv))
                    temp.append(max(_temp))
                    humidity.append(max(_humidity))
                    winddir.append(max(_winddir))
                    windspeed.append(max(_windspeed))
                    windgust.append(max(_windgust))
                    dewpoint.append(max(_dewpoint))
                    maxdailygust.append(max(_maxdailygust))
                    feelslike.append(max(_feelslike))
                    hourlyrainin.append(max(_hourlyrainin))
                    dailyrainin.append(max(_dailyrainin))
                    weeklyrainin.append(max(_weeklyrainin))
                    monthlyrainin.append(max(_monthlyrainin))
                    yearlyrainin.append(max(_yearlyrainin))
                    print('_id:',_id)
        
                __id              = []
                _loc_oid          = []
                _TIME             = []
                _solarradiation   = []
                _uv               = []
                _temp             = []
                _humidity         = []
                _winddir          = []
                _windspeed        = []
                _windgust         = []
                _dewpoint         = []
                _maxdailygust     = []
                _feelslike        = []
                _hourlyrainin     = []
                _dailyrainin      = []
                _weeklyrainin     = []
                _monthlyrainin    = []
                _yearlyrainin     = []
            
            #print("len(data) == len(__id) : ",len(data),dataline['id'], max(_id))
            #if len(data) == dataline['id']:
                
            
            #print(dataline)
            __id.append(dataline['id'])
            _loc_oid.append(dataline['loc_oid'])
            _#TIME.append(dataline['TIME'])
            _solarradiation.append(dataline['solarradiation'])
            _uv.append(dataline['uv'])
            _temp.append(dataline['temp'])
            _humidity.append(dataline['humidity'])
            _winddir.append(dataline['winddir'])
            _windspeed.append(dataline['windspeed'])
            _windgust.append(dataline['windgust'])
            _dewpoint.append(dataline['dewpoint'])
            _maxdailygust.append(dataline['maxdailygust'])
            _feelslike.append(dataline['feelslike'])
            _hourlyrainin.append(dataline['hourlyrainin'])
            _dailyrainin.append(dataline['dailyrainin'])
            _weeklyrainin.append(dataline['weeklyrainin'])
            _monthlyrainin.append(dataline['monthlyrainin'])
            _yearlyrainin.append(dataline['yearlyrainin'])
            #print(__id, max(__id))
        #print(_id)
        
    base_time = str(int(dataline['TIME'][11:13])+1).zfill(2)
    _TIME = dataline['TIME'][:10]+' '+base_time
    #base_time = dataline['TIME'][11:13]
    _id.append(max(__id))
    loc_oid.append(max(_loc_oid))
    TIME.append(_TIME)
    solarradiation.append(max(_solarradiation))
    uv.append(max(_uv))
    temp.append(max(_temp))
    humidity.append(max(_humidity))
    winddir.append(max(_winddir))
    windspeed.append(max(_windspeed))
    windgust.append(max(_windgust))
    dewpoint.append(max(_dewpoint))
    maxdailygust.append(max(_maxdailygust))
    feelslike.append(max(_feelslike))
    hourlyrainin.append(max(_hourlyrainin))
    dailyrainin.append(max(_dailyrainin))
    weeklyrainin.append(max(_weeklyrainin))
    monthlyrainin.append(max(_monthlyrainin))
    yearlyrainin.append(max(_yearlyrainin))
    
    print("-"*100)
    print(_id)
    print("-"*100)
    print("df['id]': ", df['id'])
    #print(np.shape(_id))
    print(df)
    df['id'] = _id
    df['loc_oid'] = loc_oid
    df['TIME'] = TIME
    df['solarradiation'] = solarradiation
    df['uv'] = uv
    df['temp'] = temp
    df['humidity'] = humidity
    df['winddir'] = winddir
    df['windspeed'] = windspeed
    df['windgust'] = windgust
    df['dewpoint'] = dewpoint
    df['maxdailygust'] = maxdailygust
    df['feelslike'] = feelslike
    df['hourlyrainin'] = hourlyrainin
    df['dailyrainin'] = dailyrainin
    df['weeklyrainin'] = weeklyrainin
    df['monthlyrainin'] = monthlyrainin
    df['yearlyrainin'] = yearlyrainin
    #display(df)
    #print(total_df.shape)
    #print(total_df.shape[0])
    
    total_df = pd.concat([total_df, df])#hrPow.append(filedata)
    # save csv file
total_df.to_csv(base_dir + '_sensor_df_1hour_sum.csv', index=False, header=True)

print("\n-----------------------------------------------------------------")
print("\t FILE SIZE ERROR > file path / file size")
for f in err_file_size:
    print(f)

C:/elsys/solar/_SENSOR/json\20210128.json
19854	_id: [2]
_id: [2, 8]
_id: [2, 8, 14]
_id: [2, 8, 14, 20]
_id: [2, 8, 14, 20, 26]
_id: [2, 8, 14, 20, 26, 32]
_id: [2, 8, 14, 20, 26, 32, 38]
_id: [2, 8, 14, 20, 26, 32, 38, 44]
_id: [2, 8, 14, 20, 26, 32, 38, 44, 50]
_id: [2, 8, 14, 20, 26, 32, 38, 44, 50, 56]
----------------------------------------------------------------------------------------------------
[2, 8, 14, 20, 26, 32, 38, 44, 50, 56, 62]
----------------------------------------------------------------------------------------------------
df['id]':  Series([], Name: id, dtype: object)
Empty DataFrame
Columns: [id, loc_oid, TIME, solarradiation, uv, temp, humidity, winddir, windspeed, windgust, dewpoint, maxdailygust, feelslike, hourlyrainin, dailyrainin, weeklyrainin, monthlyrainin, yearlyrainin]
Index: []
C:/elsys/solar/_SENSOR/json\20210129.json
46204	_id: [68]
_id: [68, 74]
_id: [68, 74, 80]
_id: [68, 74, 80, 86]
_id: [68, 74, 80, 86, 92]
_id: [68, 74, 80, 86, 92, 98]
_id: 

----------------------------------------------------------------------------------------------------
[2516, 2522, 2528, 2534, 2540, 2546, 2552, 2558, 2564, 2570, 2576, 2582, 2588, 2594, 2600, 2606, 2612, 2618, 2624, 2630, 2636, 2642, 2648, 2654]
----------------------------------------------------------------------------------------------------
df['id]':  Series([], Name: id, dtype: object)
Empty DataFrame
Columns: [id, loc_oid, TIME, solarradiation, uv, temp, humidity, winddir, windspeed, windgust, dewpoint, maxdailygust, feelslike, hourlyrainin, dailyrainin, weeklyrainin, monthlyrainin, yearlyrainin]
Index: []
C:/elsys/solar/_SENSOR/json\20210216.json
46124	_id: [2660]
_id: [2660, 2666]
_id: [2660, 2666, 2672]
_id: [2660, 2666, 2672, 2678]
_id: [2660, 2666, 2672, 2678, 2684]
_id: [2660, 2666, 2672, 2678, 2684, 2690]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702, 2707]
_id: [26

Index: []
C:/elsys/solar/_SENSOR/json\20210306.json
45569	_id: [5252]
_id: [5252, 5258]
_id: [5252, 5258, 5264]
_id: [5252, 5258, 5264, 5270]
_id: [5252, 5258, 5264, 5270, 5276]
_id: [5252, 5258, 5264, 5270, 5276, 5282]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318, 5324]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318, 5324, 5330]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318, 5324, 5330, 5336]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318, 5324, 5330, 5336, 5342]
_id: [5252, 5258, 5264, 5270, 

C:/elsys/solar/_SENSOR/json\20210324.json
45492	_id: [7844]
_id: [7844, 7850]
_id: [7844, 7850, 7856]
_id: [7844, 7850, 7856, 7862]
_id: [7844, 7850, 7856, 7862, 7868]
_id: [7844, 7850, 7856, 7862, 7868, 7874]
_id: [7844, 7850, 7856, 7862, 7868, 7874, 7880]
_id: [7844, 7850, 7856, 7862, 7868, 7874, 7880, 7886]
_id: [7844, 7850, 7856, 7862, 7868, 7874, 7880, 7886, 7892]
_id: [7844, 7850, 7856, 7862, 7868, 7874, 7880, 7886, 7892, 7898]
_id: [7844, 7850, 7856, 7862, 7868, 7874, 7880, 7886, 7892, 7898, 7904]
_id: [7844, 7850, 7856, 7862, 7868, 7874, 7880, 7886, 7892, 7898, 7904, 7910]
_id: [7844, 7850, 7856, 7862, 7868, 7874, 7880, 7886, 7892, 7898, 7904, 7910, 7916]
_id: [7844, 7850, 7856, 7862, 7868, 7874, 7880, 7886, 7892, 7898, 7904, 7910, 7916, 7922]
_id: [7844, 7850, 7856, 7862, 7868, 7874, 7880, 7886, 7892, 7898, 7904, 7910, 7916, 7922, 7928]
_id: [7844, 7850, 7856, 7862, 7868, 7874, 7880, 7886, 7892, 7898, 7904, 7910, 7916, 7922, 7928, 7934]
_id: [7844, 7850, 7856, 7862, 7868, 7874

45479	_id: [10436]
_id: [10436, 10442]
_id: [10436, 10442, 10448]
_id: [10436, 10442, 10448, 10454]
_id: [10436, 10442, 10448, 10454, 10460]
_id: [10436, 10442, 10448, 10454, 10460, 10466]
_id: [10436, 10442, 10448, 10454, 10460, 10466, 10472]
_id: [10436, 10442, 10448, 10454, 10460, 10466, 10472, 10478]
_id: [10436, 10442, 10448, 10454, 10460, 10466, 10472, 10478, 10484]
_id: [10436, 10442, 10448, 10454, 10460, 10466, 10472, 10478, 10484, 10490]
_id: [10436, 10442, 10448, 10454, 10460, 10466, 10472, 10478, 10484, 10490, 10496]
_id: [10436, 10442, 10448, 10454, 10460, 10466, 10472, 10478, 10484, 10490, 10496, 10502]
_id: [10436, 10442, 10448, 10454, 10460, 10466, 10472, 10478, 10484, 10490, 10496, 10502, 10508]
_id: [10436, 10442, 10448, 10454, 10460, 10466, 10472, 10478, 10484, 10490, 10496, 10502, 10508, 10514]
_id: [10436, 10442, 10448, 10454, 10460, 10466, 10472, 10478, 10484, 10490, 10496, 10502, 10508, 10514, 10520]
_id: [10436, 10442, 10448, 10454, 10460, 10466, 10472, 10478, 10

C:/elsys/solar/_SENSOR/json\20210429.json
45875	_id: [13028]
_id: [13028, 13034]
_id: [13028, 13034, 13040]
_id: [13028, 13034, 13040, 13046]
_id: [13028, 13034, 13040, 13046, 13052]
_id: [13028, 13034, 13040, 13046, 13052, 13058]
_id: [13028, 13034, 13040, 13046, 13052, 13058, 13064]
_id: [13028, 13034, 13040, 13046, 13052, 13058, 13064, 13070]
_id: [13028, 13034, 13040, 13046, 13052, 13058, 13064, 13070, 13076]
_id: [13028, 13034, 13040, 13046, 13052, 13058, 13064, 13070, 13076, 13082]
_id: [13028, 13034, 13040, 13046, 13052, 13058, 13064, 13070, 13076, 13082, 13087]
_id: [13028, 13034, 13040, 13046, 13052, 13058, 13064, 13070, 13076, 13082, 13087, 13093]
_id: [13028, 13034, 13040, 13046, 13052, 13058, 13064, 13070, 13076, 13082, 13087, 13093, 13099]
_id: [13028, 13034, 13040, 13046, 13052, 13058, 13064, 13070, 13076, 13082, 13087, 13093, 13099, 13105]
_id: [13028, 13034, 13040, 13046, 13052, 13058, 13064, 13070, 13076, 13082, 13087, 13093, 13099, 13105, 13111]
_id: [13028, 13034, 13

45609	_id: [15620]
_id: [15620, 15626]
_id: [15620, 15626, 15632]
_id: [15620, 15626, 15632, 15638]
_id: [15620, 15626, 15632, 15638, 15644]
_id: [15620, 15626, 15632, 15638, 15644, 15650]
_id: [15620, 15626, 15632, 15638, 15644, 15650, 15656]
_id: [15620, 15626, 15632, 15638, 15644, 15650, 15656, 15662]
_id: [15620, 15626, 15632, 15638, 15644, 15650, 15656, 15662, 15668]
_id: [15620, 15626, 15632, 15638, 15644, 15650, 15656, 15662, 15668, 15674]
_id: [15620, 15626, 15632, 15638, 15644, 15650, 15656, 15662, 15668, 15674, 15680]
_id: [15620, 15626, 15632, 15638, 15644, 15650, 15656, 15662, 15668, 15674, 15680, 15686]
_id: [15620, 15626, 15632, 15638, 15644, 15650, 15656, 15662, 15668, 15674, 15680, 15686, 15692]
_id: [15620, 15626, 15632, 15638, 15644, 15650, 15656, 15662, 15668, 15674, 15680, 15686, 15692, 15698]
_id: [15620, 15626, 15632, 15638, 15644, 15650, 15656, 15662, 15668, 15674, 15680, 15686, 15692, 15698, 15704]
_id: [15620, 15626, 15632, 15638, 15644, 15650, 15656, 15662, 15


-----------------------------------------------------------------
	 FILE SIZE ERROR > file path / file size
C:/elsys/solar/_SENSOR/json\20210229.json/13


In [12]:
from pandas import read_csv

date_start = '20210129'+'00'
date_end   = '20210531'+'99'

#### sensor df 전처리
sensor_df = read_csv(base_dir+'_sensor_df_1hour_sum.csv', encoding='CP949', converters={'DATE':int})

sensor_df['NEW_TIME'] = sensor_df['TIME'].str[0:4]+sensor_df['TIME'].str[5:7]+sensor_df['TIME'].str[8:10]+sensor_df['TIME'].str[11:13]
sensor_df = sensor_df[(sensor_df['NEW_TIME'].astype(str)>=date_start) & (sensor_df['NEW_TIME'].astype(str)<=date_end)]
#sensor_df = sensor_df.interpolate(method='linear')# 결측값 보간
sensor_df.insert(0, 'DATE', sensor_df['NEW_TIME'].values, True)
sensor_df.drop(['id','loc_oid', 'TIME', 'NEW_TIME'], axis=1, inplace=True)
display(sensor_df)

sensor_df.to_csv(base_dir + '_sensor_df_1hour_sum.csv', index=False, header=True)
print(sensor_df.shape)

,DATE,solarradiation,uv,temp,humidity,winddir,windspeed,windgust,dewpoint,maxdailygust,feelslike,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin
11,2021012901,0.00,0,-4.722,77.0,331,3.979,7.778,-8.128,11.086,-7.339,0.0,0.0,12.192,12.192,12.192
12,2021012902,0.00,0,-4.889,85.0,353,4.604,6.705,-7.394,11.086,-5.389,0.0,0.0,12.192,12.192,12.192
13,2021012903,0.00,0,-5.278,78.0,358,4.381,8.896,-8.500,11.086,-5.278,0.0,0.0,12.192,12.192,12.192
14,2021012904,0.00,0,-5.500,78.0,357,4.113,5.498,-8.717,11.086,-5.500,0.0,0.0,12.192,12.192,12.192
15,2021012905,0.00,0,-5.778,76.0,351,2.995,5.498,-9.317,11.086,-9.194,0.0,0.0,12.192,12.192,12.192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2957,2021053120,39.82,0,22.778,62.0,222,0.000,0.000,12.344,5.498,22.417,0.0,0.0,0.000,96.520,502.412
2958,2021053121,0.00,0,18.111,71.0,152,0.000,0.000,11.422,5.498,18.111,0.0,0.0,0.000,96.520,502.412
2959,2021053122,0.00,0,15.611,76.0,145,0.000,0.000,10.428,5.498,15.611,0.0,0.0,0.000,96.520,502.412
2960,2021053123,0.00,0,14.222,84.0,154,0.000,0.000,10.956,5.498,14.222,0.0,0.0,0.000,96.520,502.412


(2951, 16)


### 통합 csv

In [98]:
# 1시간 단위

import csv
import json
import pandas as pd
import os
import glob
import datetime
import numpy as np
# .json -> .csv


base_dir        = 'C:/elsys/solar/'
sensor_dir      = base_dir + '_SENSOR/json'
save_dir        = base_dir + '_SENSOR/csv/'

column_list     = ['id','loc_oid','TIME','solarradiation','uv','temp','humidity','winddir','windspeed','windgust','dewpoint','maxdailygust','feelslike','hourlyrainin','dailyrainin','weeklyrainin','monthlyrainin','yearlyrainin']
err_file_size = []

json_pattern    = os.path.join(sensor_dir,'*.json')
file_list       = glob.glob(json_pattern)

#df = pd.DataFrame(columns=column_list)
total_df = pd.DataFrame(columns=column_list)
for file in file_list:
    with open(file) as json_file:
        print(file)
        # parsing json file
        json_data = json.load(json_file)
        data = json_data['result']
        df = pd.DataFrame(columns=column_list)
        
        # 2. empty file check
        file_size = os.path.getsize(file)
        print(str(file_size)+'\t',end='')
        if file_size < 100:
            err_file_size.append(file + "/" + str(file_size))#err2
            continue
            
        __id              = []
        _loc_oid          = []
        _TIME             = []
        _solarradiation   = []
        _uv               = []
        _temp             = []
        _humidity         = []
        _winddir          = []
        _windspeed        = []
        _windgust         = []
        _dewpoint         = []
        _maxdailygust     = []
        _feelslike        = []
        _hourlyrainin     = []
        _dailyrainin      = []
        _weeklyrainin     = []
        _monthlyrainin    = []
        _yearlyrainin     = []
        
        _id              = []
        loc_oid          = []
        TIME             = []
        solarradiation   = []
        uv               = []
        temp             = []
        humidity         = []
        winddir          = []
        windspeed        = []
        windgust         = []
        dewpoint         = []
        maxdailygust     = []
        feelslike        = []
        hourlyrainin     = []
        dailyrainin      = []
        weeklyrainin     = []
        monthlyrainin    = []
        yearlyrainin     = []
        
        base_time = '00'
        for dataline in data:
            #print("base time: ", base_time, ", ", dataline['TIME'][11:13])
            #print("len(data): ",len(data))
            if base_time != dataline['TIME'][11:13]:
                base_time = dataline['TIME'][11:13]
                if len(__id) != 0:
                    _TIME = dataline['TIME'][:10]+' '+base_time
                    #base_time = dataline['TIME'][11:13]
                    _id.append(max(__id))
                    loc_oid.append(max(_loc_oid))
                    TIME.append(_TIME)
                    solarradiation.append(max(_solarradiation))
                    uv.append(max(_uv))
                    temp.append(max(_temp))
                    humidity.append(max(_humidity))
                    winddir.append(max(_winddir))
                    windspeed.append(max(_windspeed))
                    windgust.append(max(_windgust))
                    dewpoint.append(max(_dewpoint))
                    maxdailygust.append(max(_maxdailygust))
                    feelslike.append(max(_feelslike))
                    hourlyrainin.append(max(_hourlyrainin))
                    dailyrainin.append(max(_dailyrainin))
                    weeklyrainin.append(max(_weeklyrainin))
                    monthlyrainin.append(max(_monthlyrainin))
                    yearlyrainin.append(max(_yearlyrainin))
                    print('_id:',_id)
        
                __id              = []
                _loc_oid          = []
                _TIME             = []
                _solarradiation   = []
                _uv               = []
                _temp             = []
                _humidity         = []
                _winddir          = []
                _windspeed        = []
                _windgust         = []
                _dewpoint         = []
                _maxdailygust     = []
                _feelslike        = []
                _hourlyrainin     = []
                _dailyrainin      = []
                _weeklyrainin     = []
                _monthlyrainin    = []
                _yearlyrainin     = []
            
            #print("len(data) == len(__id) : ",len(data),dataline['id'], max(_id))
            #if len(data) == dataline['id']:
                
            
            #print(dataline)
            __id.append(dataline['id'])
            _loc_oid.append(dataline['loc_oid'])
            _#TIME.append(dataline['TIME'])
            _solarradiation.append(dataline['solarradiation'])
            _uv.append(dataline['uv'])
            _temp.append(dataline['temp'])
            _humidity.append(dataline['humidity'])
            _winddir.append(dataline['winddir'])
            _windspeed.append(dataline['windspeed'])
            _windgust.append(dataline['windgust'])
            _dewpoint.append(dataline['dewpoint'])
            _maxdailygust.append(dataline['maxdailygust'])
            _feelslike.append(dataline['feelslike'])
            _hourlyrainin.append(dataline['hourlyrainin'])
            _dailyrainin.append(dataline['dailyrainin'])
            _weeklyrainin.append(dataline['weeklyrainin'])
            _monthlyrainin.append(dataline['monthlyrainin'])
            _yearlyrainin.append(dataline['yearlyrainin'])
            #print(__id, max(__id))
        #print(_id)
        
    base_time = str(int(dataline['TIME'][11:13])+1).zfill(2)
    _TIME = dataline['TIME'][:10]+' '+base_time
    #base_time = dataline['TIME'][11:13]
    _id.append(max(__id))
    loc_oid.append(max(_loc_oid))
    TIME.append(_TIME)
    solarradiation.append(max(_solarradiation))
    uv.append(max(_uv))
    temp.append(max(_temp))
    humidity.append(max(_humidity))
    winddir.append(max(_winddir))
    windspeed.append(max(_windspeed))
    windgust.append(max(_windgust))
    dewpoint.append(max(_dewpoint))
    maxdailygust.append(max(_maxdailygust))
    feelslike.append(max(_feelslike))
    hourlyrainin.append(max(_hourlyrainin))
    dailyrainin.append(max(_dailyrainin))
    weeklyrainin.append(max(_weeklyrainin))
    monthlyrainin.append(max(_monthlyrainin))
    yearlyrainin.append(max(_yearlyrainin))
    
    print("-"*100)
    print(_id)
    print("-"*100)
    print("df['id]': ", df['id'])
    #print(np.shape(_id))
    print(df)
    df['id'] = _id
    df['loc_oid'] = loc_oid
    df['TIME'] = TIME
    df['solarradiation'] = solarradiation
    df['uv'] = uv
    df['temp'] = temp
    df['humidity'] = humidity
    df['winddir'] = winddir
    df['windspeed'] = windspeed
    df['windgust'] = windgust
    df['dewpoint'] = dewpoint
    df['maxdailygust'] = maxdailygust
    df['feelslike'] = feelslike
    df['hourlyrainin'] = hourlyrainin
    df['dailyrainin'] = dailyrainin
    df['weeklyrainin'] = weeklyrainin
    df['monthlyrainin'] = monthlyrainin
    df['yearlyrainin'] = yearlyrainin
    #display(df)
    #print(total_df.shape)
    #print(total_df.shape[0])
    
    total_df = pd.concat([total_df, df])#hrPow.append(filedata)
    # save csv file
total_df.to_csv(base_dir + '_sensor_df_1hour.csv', index=False, header=True)

print("\n-----------------------------------------------------------------")
print("\t FILE SIZE ERROR > file path / file size")
for f in err_file_size:
    print(f)

C:/elsys/solar/_SENSOR/json\20210128.json
19854	_id: [2]
_id: [2, 8]
_id: [2, 8, 14]
_id: [2, 8, 14, 20]
_id: [2, 8, 14, 20, 26]
_id: [2, 8, 14, 20, 26, 32]
_id: [2, 8, 14, 20, 26, 32, 38]
_id: [2, 8, 14, 20, 26, 32, 38, 44]
_id: [2, 8, 14, 20, 26, 32, 38, 44, 50]
_id: [2, 8, 14, 20, 26, 32, 38, 44, 50, 56]
----------------------------------------------------------------------------------------------------
[2, 8, 14, 20, 26, 32, 38, 44, 50, 56, 62]
----------------------------------------------------------------------------------------------------
df['id]':  Series([], Name: id, dtype: object)
Empty DataFrame
Columns: [id, loc_oid, TIME, solarradiation, uv, temp, humidity, winddir, windspeed, windgust, dewpoint, maxdailygust, feelslike, hourlyrainin, dailyrainin, weeklyrainin, monthlyrainin, yearlyrainin]
Index: []
C:/elsys/solar/_SENSOR/json\20210129.json
46204	_id: [68]
_id: [68, 74]
_id: [68, 74, 80]
_id: [68, 74, 80, 86]
_id: [68, 74, 80, 86, 92]
_id: [68, 74, 80, 86, 92, 98]
_id: 

Index: []
C:/elsys/solar/_SENSOR/json\20210216.json
46124	_id: [2660]
_id: [2660, 2666]
_id: [2660, 2666, 2672]
_id: [2660, 2666, 2672, 2678]
_id: [2660, 2666, 2672, 2678, 2684]
_id: [2660, 2666, 2672, 2678, 2684, 2690]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702, 2707]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702, 2707, 2713]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702, 2707, 2713, 2719]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702, 2707, 2713, 2719, 2725]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702, 2707, 2713, 2719, 2725, 2731]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702, 2707, 2713, 2719, 2725, 2731, 2737]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702, 2707, 2713, 2719, 2725, 2731, 2737, 2743]
_id: [2660, 2666, 2672, 2678, 2684, 2690, 2696, 2702, 2707, 2713, 2719, 2725, 2731, 2737, 2743, 2749]
_id: [2660, 2666, 2672, 2678, 

45569	_id: [5252]
_id: [5252, 5258]
_id: [5252, 5258, 5264]
_id: [5252, 5258, 5264, 5270]
_id: [5252, 5258, 5264, 5270, 5276]
_id: [5252, 5258, 5264, 5270, 5276, 5282]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318, 5324]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318, 5324, 5330]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318, 5324, 5330, 5336]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318, 5324, 5330, 5336, 5342]
_id: [5252, 5258, 5264, 5270, 5276, 5282, 5288, 5294, 5300, 5306, 5312, 5318, 5324

C:/elsys/solar/_SENSOR/json\20210325.json
45403	_id: [7988]
_id: [7988, 7994]
_id: [7988, 7994, 8000]
_id: [7988, 7994, 8000, 8006]
_id: [7988, 7994, 8000, 8006, 8012]
_id: [7988, 7994, 8000, 8006, 8012, 8018]
_id: [7988, 7994, 8000, 8006, 8012, 8018, 8024]
_id: [7988, 7994, 8000, 8006, 8012, 8018, 8024, 8030]
_id: [7988, 7994, 8000, 8006, 8012, 8018, 8024, 8030, 8036]
_id: [7988, 7994, 8000, 8006, 8012, 8018, 8024, 8030, 8036, 8042]
_id: [7988, 7994, 8000, 8006, 8012, 8018, 8024, 8030, 8036, 8042, 8048]
_id: [7988, 7994, 8000, 8006, 8012, 8018, 8024, 8030, 8036, 8042, 8048, 8054]
_id: [7988, 7994, 8000, 8006, 8012, 8018, 8024, 8030, 8036, 8042, 8048, 8054, 8060]
_id: [7988, 7994, 8000, 8006, 8012, 8018, 8024, 8030, 8036, 8042, 8048, 8054, 8060, 8066]
_id: [7988, 7994, 8000, 8006, 8012, 8018, 8024, 8030, 8036, 8042, 8048, 8054, 8060, 8066, 8072]
_id: [7988, 7994, 8000, 8006, 8012, 8018, 8024, 8030, 8036, 8042, 8048, 8054, 8060, 8066, 8072, 8078]
_id: [7988, 7994, 8000, 8006, 8012, 8018

46393	_id: [10580]
_id: [10580, 10586]
_id: [10580, 10586, 10592]
_id: [10580, 10586, 10592, 10598]
_id: [10580, 10586, 10592, 10598, 10604]
_id: [10580, 10586, 10592, 10598, 10604, 10610]
_id: [10580, 10586, 10592, 10598, 10604, 10610, 10616]
_id: [10580, 10586, 10592, 10598, 10604, 10610, 10616, 10622]
_id: [10580, 10586, 10592, 10598, 10604, 10610, 10616, 10622, 10628]
_id: [10580, 10586, 10592, 10598, 10604, 10610, 10616, 10622, 10628, 10633]
_id: [10580, 10586, 10592, 10598, 10604, 10610, 10616, 10622, 10628, 10633, 10639]
_id: [10580, 10586, 10592, 10598, 10604, 10610, 10616, 10622, 10628, 10633, 10639, 10645]
_id: [10580, 10586, 10592, 10598, 10604, 10610, 10616, 10622, 10628, 10633, 10639, 10645, 10651]
_id: [10580, 10586, 10592, 10598, 10604, 10610, 10616, 10622, 10628, 10633, 10639, 10645, 10651, 10657]
_id: [10580, 10586, 10592, 10598, 10604, 10610, 10616, 10622, 10628, 10633, 10639, 10645, 10651, 10657, 10663]
_id: [10580, 10586, 10592, 10598, 10604, 10610, 10616, 10622, 10

Index: []
C:/elsys/solar/_SENSOR/json\20210501.json
46592	_id: [13316]
_id: [13316, 13322]
_id: [13316, 13322, 13328]
_id: [13316, 13322, 13328, 13334]
_id: [13316, 13322, 13328, 13334, 13340]
_id: [13316, 13322, 13328, 13334, 13340, 13346]
_id: [13316, 13322, 13328, 13334, 13340, 13346, 13352]
_id: [13316, 13322, 13328, 13334, 13340, 13346, 13352, 13358]
_id: [13316, 13322, 13328, 13334, 13340, 13346, 13352, 13358, 13364]
_id: [13316, 13322, 13328, 13334, 13340, 13346, 13352, 13358, 13364, 13370]
_id: [13316, 13322, 13328, 13334, 13340, 13346, 13352, 13358, 13364, 13370, 13376]
_id: [13316, 13322, 13328, 13334, 13340, 13346, 13352, 13358, 13364, 13370, 13376, 13382]
_id: [13316, 13322, 13328, 13334, 13340, 13346, 13352, 13358, 13364, 13370, 13376, 13382, 13388]
_id: [13316, 13322, 13328, 13334, 13340, 13346, 13352, 13358, 13364, 13370, 13376, 13382, 13388, 13394]
_id: [13316, 13322, 13328, 13334, 13340, 13346, 13352, 13358, 13364, 13370, 13376, 13382, 13388, 13394, 13400]
_id: [13316,

45462	_id: [15906]
_id: [15906, 15912]
_id: [15906, 15912, 15918]
_id: [15906, 15912, 15918, 15924]
_id: [15906, 15912, 15918, 15924, 15930]
_id: [15906, 15912, 15918, 15924, 15930, 15936]
_id: [15906, 15912, 15918, 15924, 15930, 15936, 15942]
_id: [15906, 15912, 15918, 15924, 15930, 15936, 15942, 15948]
_id: [15906, 15912, 15918, 15924, 15930, 15936, 15942, 15948, 15954]
_id: [15906, 15912, 15918, 15924, 15930, 15936, 15942, 15948, 15954, 15960]
_id: [15906, 15912, 15918, 15924, 15930, 15936, 15942, 15948, 15954, 15960, 15966]
_id: [15906, 15912, 15918, 15924, 15930, 15936, 15942, 15948, 15954, 15960, 15966, 15972]
_id: [15906, 15912, 15918, 15924, 15930, 15936, 15942, 15948, 15954, 15960, 15966, 15972, 15978]
_id: [15906, 15912, 15918, 15924, 15930, 15936, 15942, 15948, 15954, 15960, 15966, 15972, 15978, 15984]
_id: [15906, 15912, 15918, 15924, 15930, 15936, 15942, 15948, 15954, 15960, 15966, 15972, 15978, 15984, 15990]
_id: [15906, 15912, 15918, 15924, 15930, 15936, 15942, 15948, 15

In [107]:
from pandas import read_csv

date_start = '20210129'+'00'
date_end   = '20210531'+'99'

#### sensor df 전처리
sensor_df = read_csv(base_dir+'_sensor_df_1hour.csv', encoding='CP949', converters={'DATE':int})

sensor_df['NEW_TIME'] = sensor_df['TIME'].str[0:4]+sensor_df['TIME'].str[5:7]+sensor_df['TIME'].str[8:10]+sensor_df['TIME'].str[11:13]
sensor_df = sensor_df[(sensor_df['NEW_TIME'].astype(str)>=date_start) & (sensor_df['NEW_TIME'].astype(str)<=date_end)]
#sensor_df = sensor_df.interpolate(method='linear')# 결측값 보간
sensor_df.insert(0, 'DATE', sensor_df['NEW_TIME'].values, True)
sensor_df.drop(['id','loc_oid', 'TIME', 'NEW_TIME'], axis=1, inplace=True)
display(sensor_df)

sensor_df.to_csv(base_dir + '_sensor_df_1hour.csv', index=False, header=True)

,DATE,solarradiation,uv,temp,humidity,winddir,windspeed,windgust,dewpoint,maxdailygust,feelslike,hourlyrainin,dailyrainin,weeklyrainin,monthlyrainin,yearlyrainin
11,2021012901,0.00,0,-4.722,77.0,331,3.979,7.778,-8.128,11.086,-7.339,0.0,0.0,12.192,12.192,12.192
12,2021012902,0.00,0,-4.889,85.0,353,4.604,6.705,-7.394,11.086,-5.389,0.0,0.0,12.192,12.192,12.192
13,2021012903,0.00,0,-5.278,78.0,358,4.381,8.896,-8.500,11.086,-5.278,0.0,0.0,12.192,12.192,12.192
14,2021012904,0.00,0,-5.500,78.0,357,4.113,5.498,-8.717,11.086,-5.500,0.0,0.0,12.192,12.192,12.192
15,2021012905,0.00,0,-5.778,76.0,351,2.995,5.498,-9.317,11.086,-9.194,0.0,0.0,12.192,12.192,12.192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2957,2021053120,39.82,0,22.778,62.0,222,0.000,0.000,12.344,5.498,22.417,0.0,0.0,0.000,96.520,502.412
2958,2021053121,0.00,0,18.111,71.0,152,0.000,0.000,11.422,5.498,18.111,0.0,0.0,0.000,96.520,502.412
2959,2021053122,0.00,0,15.611,76.0,145,0.000,0.000,10.428,5.498,15.611,0.0,0.0,0.000,96.520,502.412
2960,2021053123,0.00,0,14.222,84.0,154,0.000,0.000,10.956,5.498,14.222,0.0,0.0,0.000,96.520,502.412


## 개별 CSV

In [19]:
import csv
import json
import pandas as pd
import os
import glob
import datetime
# .json -> .csv


base_dir        = 'C:/elsys/solar/'
sensor_dir      = base_dir + '_SENSOR/json'
save_dir        = base_dir + '_SENSOR/csv/'

column_list     = ['id','loc_oid','TIME','solarradiation','uv','temp','humidity','winddir','windspeed','windgust','dewpoint','maxdailygust','feelslike','hourlyrainin','dailyrainin','weeklyrainin','monthlyrainin','yearlyrainin']
err_file_size = []

json_pattern    = os.path.join(sensor_dir,'*.json')
file_list       = glob.glob(json_pattern)


for file in file_list:
    with open(file) as json_file:
        print(file)
        # parsing json file
        json_data = json.load(json_file)
        data = json_data['result']

        
        # 2. empty file check
        file_size = os.path.getsize(file)
        print(str(file_size)+'\t',end='')
        if file_size < 100:
            err_file_size.append(file + "/" + str(file_size))#err2
            continue
            
        _id              = []
        loc_oid          = []
        TIME             = []
        solarradiation   = []
        uv               = []
        temp             = []
        humidity         = []
        winddir          = []
        windspeed        = []
        windgust         = []
        dewpoint         = []
        maxdailygust     = []
        feelslike        = []
        hourlyrainin     = []
        dailyrainin      = []
        weeklyrainin     = []
        monthlyrainin    = []
        yearlyrainin     = []
        
        if 144 != len(data):
            print(len(data))
            
        for dataline in data:
            #print(dataline)
            _id.append(dataline['id'])
            loc_oid.append(dataline['loc_oid'])
            TIME.append(dataline['TIME'])
            solarradiation.append(dataline['solarradiation'])
            uv.append(dataline['uv'])
            temp.append(dataline['temp'])
            humidity.append(dataline['humidity'])
            winddir.append(dataline['winddir'])
            windspeed.append(dataline['windspeed'])
            windgust.append(dataline['windgust'])
            dewpoint.append(dataline['dewpoint'])
            maxdailygust.append(dataline['maxdailygust'])
            feelslike.append(dataline['feelslike'])
            hourlyrainin.append(dataline['hourlyrainin'])
            dailyrainin.append(dataline['dailyrainin'])
            weeklyrainin.append(dataline['weeklyrainin'])
            monthlyrainin.append(dataline['monthlyrainin'])
            yearlyrainin.append(dataline['yearlyrainin'])
            
            
                
        df = pd.DataFrame(columns=column_list)
        df['id'] = _id
        df['loc_oid'] = loc_oid
        df['TIME'] = TIME
        df['solarradiation'] = solarradiation
        df['uv'] = uv
        df['temp'] = temp
        df['humidity'] = humidity
        df['winddir'] = winddir
        df['windspeed'] = windspeed
        df['windgust'] = windgust
        df['dewpoint'] = dewpoint
        df['maxdailygust'] = maxdailygust
        df['feelslike'] = feelslike
        df['hourlyrainin'] = hourlyrainin
        df['dailyrainin'] = dailyrainin
        df['weeklyrainin'] = weeklyrainin
        df['monthlyrainin'] = monthlyrainin
        df['yearlyrainin'] = yearlyrainin
        
        # save csv file
        df.to_csv(save_dir + file[-13:-5] + '.csv', index=False, header=True)

print("\n-----------------------------------------------------------------")
print("\t FILE SIZE ERROR > file path / file size")
for f in err_file_size:
    print(f)

C:/elsys/solar/_SENSOR/json\20210128.json
19854	62
C:/elsys/solar/_SENSOR/json\20210129.json
46204	C:/elsys/solar/_SENSOR/json\20210130.json
45504	C:/elsys/solar/_SENSOR/json\20210131.json
44929	C:/elsys/solar/_SENSOR/json\20210201.json
45565	C:/elsys/solar/_SENSOR/json\20210202.json
45814	143
C:/elsys/solar/_SENSOR/json\20210203.json
45920	145
C:/elsys/solar/_SENSOR/json\20210204.json
46069	C:/elsys/solar/_SENSOR/json\20210205.json
45619	C:/elsys/solar/_SENSOR/json\20210206.json
45268	143
C:/elsys/solar/_SENSOR/json\20210207.json
45299	C:/elsys/solar/_SENSOR/json\20210208.json
45961	C:/elsys/solar/_SENSOR/json\20210209.json
45130	143
C:/elsys/solar/_SENSOR/json\20210210.json
45541	145
C:/elsys/solar/_SENSOR/json\20210211.json
45106	C:/elsys/solar/_SENSOR/json\20210212.json
44847	143
C:/elsys/solar/_SENSOR/json\20210213.json
45514	145
C:/elsys/solar/_SENSOR/json\20210214.json
45207	C:/elsys/solar/_SENSOR/json\20210215.json
45785	C:/elsys/solar/_SENSOR/json\20210216.json
46124	C:/elsys/